In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

# 키워드와 검색 기간 설정
keywords = [
    "검색어",
]
start_date = "20240601"
end_date = "20240831"

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)
seen_urls = set()  # 중복 URL을 저장하기 위한 집합

def fetch_search_results(keyword, start_date, end_date):
    base_url = "https://www.nongmin.com/searchMain?searchTerm="
    results = []

    page = 1
    while True:
        print(f"{keyword} {page} 페이지 수집 중...")
        search_url = base_url + urllib.parse.quote(keyword) + f"&searchType=NOTALL&searchTab=NEWS&searchRange=ALL&listNum=100&notSearchTerm=&reSearchTerm=&filterParam={start_date}~{end_date}&reporterNm=&searchChk=&orderParam=POST_SORT_ISSUE_DATE_DESC&pageNum={page}"
        driver.get(search_url)
        

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 기사 목록 가져오기
        article_list = soup.find("ul", class_="article_list")
        if not article_list:
                break
        
        articles = article_list.find_all("li")

        if not articles:
            print(f"기사가 없습니다. 페이지: {page}")
            break

        # 기사 정보 추출
        for item in articles:
            try:
                url = item.a['href'] if item.a else ""
                title = item.select_one('.tit').text.strip() if item.select_one('.tit') else ""
                date = item.select_one('.date').text.strip() if item.select_one('.date') else ""

                # 중복 URL 체크
                if url in seen_urls:
                    continue  # 이미 처리된 URL은 무시
                seen_urls.add(url)  # 새로운 URL 저장
                
                results.append({
                    'media': '농민신문', 
                    'title': title, 
                    'date': date, 
                    'section': '',  # 섹션 정보 없음
                    'article_type': '',  # 기사 유형 정보 없음
                    'writer': '', 
                    'page': '', 
                    'url': url, 
                    'content': ''  # 컨텐츠 정보 없음
                })
            except Exception as e:
                print(f"Error extracting article information: {e}")

        page += 1  # 다음 페이지로 이동
    
    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df.to_excel(filename, index=False)

all_results = []

try:
    for keyword in keywords:
        results = fetch_search_results(keyword, start_date, end_date)
        all_results.extend(results)

    # 중복된 기사 제거 (url 기준)
    df = pd.DataFrame(all_results)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]

    # 기사 수 출력
    print(f"Total number of articles: {len(df)}")

    # 중복 제거된 데이터 저장
    save_to_excel(df, f'3농민신문 {start_date}~{end_date}.xlsx')

finally:
    # 드라이버 종료
    driver.quit()

In [ ]:
# 농민일보

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문을 스크랩하는 함수 정의
def get_content_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        subtitle_tag = soup.find('div', class_='news_sub_tit')
        article_body = soup.find('div', class_='news_txt ck-content')

        if article_body:

            subtitle = ""
            if subtitle_tag:
                subtitle = subtitle_tag.get_text(separator='\n', strip=True)
                subtitle_tag.decompose()  # 서브 타이틀 제거
            # 'caption' 클래스를 가진 모든 태그를 제거합니다.
            for caption in article_body.find_all('figure', class_='image'):
                caption.decompose()

            # 'font-size:13px;padding:8px 10px 7px;color:#666;background:#f5f5f5;' 스타일을 가진 태그 제거
            for element in article_body.find_all(True, style=lambda value: value and 'font-size:13px' in value):
                element.decompose()
            content = article_body.get_text(separator='\n', strip=True)               
        else:
            content = ""        

        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 content 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_from_url(row['url'])
    if content is None:
        error_log.append(row['url'])
    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 오류 로그 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")
